In [1]:
# !pip install datasets transformers
!pip install transformers[torch]
!pip install datasets transformers==4.28.0

# !apt install git-lfs
!pip install git-lfs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 76.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 98.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.9 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.0
    Uninstalling tokenizers-0.15.0:
      Successfully uninstalled tokenizers-0.15.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


In [2]:
import transformers
import os
import json
import pandas as pd

In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [4]:
path = "/content/abcd_v1.1.json"

file = open(path)
dictionary = json.load(file)

print(dictionary.keys())

dict_keys(['train', 'dev', 'test'])


In [5]:
print(dictionary['test'][:10])

[{'convo_id': 4989, 'scenario': {'personal': {'customer_name': 'chloe zhang', 'member_level': 'gold', 'phone': '(741) 062-3341'}, 'order': {'street_address': '4247 lincoln ave', 'full_address': '4247 lincoln ave  newark, ca 82636', 'city': 'newark', 'state': 'ca', 'zip_code': '82636'}, 'product': {'names': [], 'amounts': []}, 'flow': 'storewide_query', 'subflow': 'timing_4'}, 'original': [['agent', 'Hello. How can i help you today?'], ['customer', 'Hi.  My name is Chloe Zhang.  I am curious as to when my promo code expires.'], ['customer', 'Would you be able to tell me?'], ['agent', 'Yes let me look into this im sure we can find a solution'], ['action', 'Searching the FAQ pages ...'], ['action', 'System Action: search timing'], ['agent', "Okay I've looked into this for you"], ['agent', 'Promo codes will expire 7 days after they are issued.'], ['customer', 'Ok.'], ['agent', 'Is there anything else I can help you with?'], ['customer', 'No that is all for today.  Thank you.']], 'delexed':

In [6]:
print(dictionary['test'][0]['original']) # original holds all convos

[['agent', 'Hello. How can i help you today?'], ['customer', 'Hi.  My name is Chloe Zhang.  I am curious as to when my promo code expires.'], ['customer', 'Would you be able to tell me?'], ['agent', 'Yes let me look into this im sure we can find a solution'], ['action', 'Searching the FAQ pages ...'], ['action', 'System Action: search timing'], ['agent', "Okay I've looked into this for you"], ['agent', 'Promo codes will expire 7 days after they are issued.'], ['customer', 'Ok.'], ['agent', 'Is there anything else I can help you with?'], ['customer', 'No that is all for today.  Thank you.']]


In [7]:
for word in dictionary['test'][0]['original']:
    print(word)

['agent', 'Hello. How can i help you today?']
['customer', 'Hi.  My name is Chloe Zhang.  I am curious as to when my promo code expires.']
['customer', 'Would you be able to tell me?']
['agent', 'Yes let me look into this im sure we can find a solution']
['action', 'Searching the FAQ pages ...']
['action', 'System Action: search timing']
['agent', "Okay I've looked into this for you"]
['agent', 'Promo codes will expire 7 days after they are issued.']
['customer', 'Ok.']
['agent', 'Is there anything else I can help you with?']
['customer', 'No that is all for today.  Thank you.']


In [8]:
# compiling all convos into a list
convos = []
for i in range(0,len(dictionary['test'])):
    convo = ""
    for sub in dictionary['test'][i]['original']:
        if sub[0] == "customer":
            convo += f"Customer: {sub[1]} "
        if sub[0] == "agent":
            convo += f"Agent: {sub[1]} "
    convos.append(convo)
    print(convo, end="\n\n")

Agent: Hello. How can i help you today? Customer: Hi.  My name is Chloe Zhang.  I am curious as to when my promo code expires. Customer: Would you be able to tell me? Agent: Yes let me look into this im sure we can find a solution Agent: Okay I've looked into this for you Agent: Promo codes will expire 7 days after they are issued. Customer: Ok. Agent: Is there anything else I can help you with? Customer: No that is all for today.  Thank you. 

Agent: Hi! How may I help you? Customer: Hello. I recently signed up for a subscription but it looks like you guys charged me twice for it. Agent: I see, let's fix that. May I have your full name, account and order IDs? Customer: sure, it's Albert Sanders and my account id is U85ALFLG2Z Agent: Do you have an order ID? Customer: yes its 7149958247 Agent: Thank you, what is your membership status? Customer: silver Agent: I see, and what was the extra charge amount? Customer: it was 20 dollars Agent: I'm sorry, our system is telling me this was not

In [9]:
res = []
for i in range(0,len(dictionary['test'])):
    context = ""
    for sub in dictionary['test'][i]['original']:
        if sub[0] == "agent":
            # context += "Agent: "
            response = f"{sub[1]} "
            res.append((context, response))
            context += response
        if sub[0] == "customer":
            context += f"Customer: {sub[1]} "

In [10]:
df = pd.DataFrame({
    "context": [r[0] for r in res],
    "response": [r[1] for r in res]
})

df.head(10)

,context,response
0,,Hello. How can i help you today?
1,Hello. How can i help you today? Customer: Hi....,Yes let me look into this im sure we can find ...
2,Hello. How can i help you today? Customer: Hi....,Okay I've looked into this for you
3,Hello. How can i help you today? Customer: Hi....,Promo codes will expire 7 days after they are ...
4,Hello. How can i help you today? Customer: Hi....,Is there anything else I can help you with?
5,,Hi! How may I help you?
6,Hi! How may I help you? Customer: Hello. I rec...,"I see, let's fix that. May I have your full na..."
7,Hi! How may I help you? Customer: Hello. I rec...,Do you have an order ID?
8,Hi! How may I help you? Customer: Hello. I rec...,"Thank you, what is your membership status?"
9,Hi! How may I help you? Customer: Hello. I rec...,"I see, and what was the extra charge amount?"


In [11]:
df.to_csv("testing_data.csv")